In [1]:
# import os
# os.mkdir("S8")
# with open('model.py', 'w') as fp: 
#     pass
# %cd ..
# %cd ..
import shutil
try:
  shutil.rmtree("./EVA5")
except:
  pass

In [2]:
!git clone https://github.com/abishek-raju/EVA5.git

Cloning into 'EVA5'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (81/81), done.


In [3]:
# %ls

In [4]:
%cd EVA5/S9

/content/EVA5/S9


In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.backends.cudnn as cudnn
import numpy as np
import matplotlib.pyplot as plt

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
device

'cuda'

In [8]:
from dataloader import load_cifar_data

In [9]:
trn_loader ,test_loader, classes = load_cifar_data()

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
from model import ResNet18

In [11]:
net = ResNet18(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [13]:
from train import train
from test import test

In [14]:
def save_checkpoint(checkpoint,file_name = "model.pth.tar"):
  print(f"Saving model to {file_name}")
  torch.save(checkpoint,file_name)


In [15]:
def load_checkpoint(file_name):
  print(f"Loading checkpoint from {file_name}")
  checkpoint = torch.load(file_name)
  net.load_state_dict(checkpoint["state-dict"])
  optimizer.load_state_dict(checkpoint["optimizer"])

In [16]:
# best_accuracy = 0
# for epoch in range(0,40):
#     train_average_loss,train_accuracy = train(epoch,net,device,trn_loader,optimizer,criterion)
#     test_average_loss,test_accuracy = test(epoch,net,test_loader,device,criterion)
#     print(f'\nEpoch: {epoch}\n\t Train set: Average loss: {train_average_loss}, Accuracy: {train_accuracy}%')
#     print(f'\n\tTest set: Average loss: {test_average_loss}, Accuracy: {test_accuracy}%')
#     if test_accuracy > best_accuracy:
#       best_accuracy = test_accuracy
#       checkpoint = {'state-dict':net.state_dict(),'optimizer':optimizer.state_dict()}
#       save_checkpoint(checkpoint)
#     scheduler.step()

In [17]:
load_checkpoint("./model.pth.tar")

Loading checkpoint from ./model.pth.tar


In [18]:
try:
  shutil.rmtree("/content/EVA5/S9/cifar_test_single")
except:
  pass

In [19]:
import shutil
import os
destination = shutil.copytree("/content/EVA5/S9/cifar_test", "/content/EVA5/S9/cifar_test_single") 
# os.remove("/content/EVA5/S9/cifar_test_single/just_a_folder/cars_1.jpeg")
os.remove("/content/EVA5/S9/cifar_test_single/just_a_folder/cars_2.jpeg")
os.remove("/content/EVA5/S9/cifar_test_single/just_a_folder/plane_1.jpg")
os.remove("/content/EVA5/S9/cifar_test_single/just_a_folder/plane_2.jpeg")

In [81]:
net.to(device)
net.eval()
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    transforms.Resize((32,32))])
transform_test_without_resize = transforms.Compose([
    transforms.ToTensor()])
# dataset = {'predict' : datasets.ImageFolder("/content/EVA5/S9/cifar_test_single", transform_test)}
# dataloader = {'predict': torch.utils.data.DataLoader(dataset['predict'], batch_size = 1, shuffle=False, num_workers=4)}

# outputs = list()
# for inputs, labels in dataloader['predict']:
#     inputs = inputs.to(device)
#     output = net(inputs)
#     output = output.to(device)
#     index = output.cpu().data.numpy().argmax()
#     print(index)

In [97]:
from PIL import Image
from torch.autograd import Variable
def image_loader(image_name,device):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    original_image = Image.open(image_name)
    # original_image = transform_test_without_resize(original_image)
    image = transform_test(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.to(device), original_image #assumes that you're using GPU

In [96]:
# temp = np.array(orgi)
# temp.shape

(168, 299, 3)

In [21]:
# net

In [22]:
# net.module.layer4[1].conv2

In [23]:
# res50_model.module

In [24]:
# temp = nn.Sequential(*list(net.children()))

In [25]:
# outputs= []
# def hook(module, input, output):
#     outputs.append(output)

# res50_model = net
# res50_model.module.layer4[0].conv2.register_forward_hook(hook)
# out = res50_model(res)
# out = res50_model(res1)
# print(outputs)

In [26]:
# list(net.children())[0].children

In [27]:
# nn.Sequential(*list(original_model.children())[:-2])

In [28]:
# nn.Sequential(*list(net.module.children()))

In [52]:
class Grad_Res(nn.Module):
    def __init__(self,model):
        super().__init__()
        
        # get the pretrained VGG19 network
        self.res_model = model
        
        # disect the network to access its last convolutional layer
        self.features_conv = nn.Sequential(*list(self.res_model.module.children())[:-2])
        # self.features_conv = self.vgg.features[:36]
        
        # # get the max pool of the features stem
        # self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        
        self.classifier1 = list(self.res_model.module.children())[-2:][0]
        self.classifier2 = list(self.res_model.module.children())[-2:][1]

        
        # placeholder for the gradients
        self.gradients = None
    
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        x = self.features_conv(x)
        # print("aaaaaa")
        # register the hook
        h = x.register_hook(self.activations_hook)
        # print("bbbbbb")
        print(x.shape)
        # apply the remaining pooling
        x = self.classifier1(x)
        # print("before linear",x.shape)
        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)
        x = self.classifier2(x)
        # print("cccccc")
        return x
    
    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients
    
    # method for the activation exctraction
    def get_activations(self, x):
        return self.features_conv(x)

In [103]:
def grad_cam_view(class_of_interest,img,device):
  grad_res = Grad_Res(net)
  grad_res.eval()
  img = img.to(device)
  pred = grad_res(img)
  pred[:,class_of_interest].backward()
  gradients = grad_res.get_activations_gradient()
  pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
  activations = grad_res.get_activations(img).detach()
  for i in range(256):
    activations[:, i, :, :] *= pooled_gradients[i]
  heatmap = torch.mean(activations, dim=1).squeeze()
  heatmap = np.maximum(heatmap.cpu(), 0)
  heatmap /= torch.max(heatmap)
  return heatmap
# classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')
# img, _ = next(iter(dataloader['predict']))
# heatmap_generated = grad_cam_view(1,img,device)

torch.Size([1, 256, 8, 8])


In [102]:
# img,orgi = image_loader("/content/EVA5/S9/cifar_test/just_a_folder/plane_1.jpg",device)


In [121]:

img,orgi = image_loader("/content/EVA5/S9/cifar_test/just_a_folder/cars_1.jpeg",device)
# classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')

heatmap_generated = grad_cam_view(1,img,device)
import cv2
# temp = cv2.imread('/content/EVA5/S9/cifar_test_single/just_a_folder/cars_1.jpeg')
# temp = orgi.permute(1, 2, 0).numpy()
temp = np.array(orgi)
# temp = Image.fromarray(temp)
heatmap = cv2.resize(heatmap_generated.numpy(), (temp.shape[1], temp.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.7+ Image.fromarray(temp)
# superimposed_img_resized = cv2.resize(superimposed_img, (200, 200))
cv2.imwrite('./map.jpg', superimposed_img)

torch.Size([1, 256, 8, 8])


True

In [ ]:
# # initialize the VGG model
# vgg = VGG(net)

# # set the evaluation mode
# vgg.eval()

# # get the image from the dataloader
# img, _ = next(iter(dataloader['predict']))
# img = img.to(device)
# # get the most likely prediction of the model
# pred = vgg(img)

In [ ]:
# pred

In [ ]:
# pred[:,1].backward()

In [ ]:
# gradients = vgg.get_activations_gradient()

In [ ]:
# pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

In [ ]:
# activations = vgg.get_activations(img).detach()

In [ ]:
# pooled_gradients.shape

In [ ]:
# activations.shape

In [ ]:
# for i in range(256):
#     activations[:, i, :, :] *= pooled_gradients[i]

In [ ]:
# heatmap = torch.mean(activations, dim=1).squeeze()

In [ ]:
# import numpy as np
# np.maximum(heatmap.cpu(), 0)

In [ ]:
# heatmap = np.maximum(heatmap.cpu(), 0)

In [ ]:
# heatmap /= torch.max(heatmap)

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# plt.matshow(heatmap.squeeze())

In [ ]:
# img

In [ ]:
# plt.matshow(np.transpose(img[0].cpu(),(1,2,0)))

In [ ]:
# heatmap.numpy()

In [91]:
orgi

tensor([[[0.4353, 0.4353, 0.4314,  ..., 0.8588, 0.8588, 0.8588],
         [0.4353, 0.4314, 0.4314,  ..., 0.8627, 0.8627, 0.8627],
         [0.4510, 0.4510, 0.4471,  ..., 0.8549, 0.8549, 0.8549],
         ...,
         [0.4235, 0.4157, 0.3412,  ..., 0.5059, 0.5098, 0.5098],
         [0.4353, 0.4510, 0.4157,  ..., 0.5412, 0.5451, 0.5412],
         [0.4588, 0.3725, 0.4392,  ..., 0.6196, 0.6235, 0.6157]],

        [[0.6078, 0.6078, 0.6039,  ..., 0.9020, 0.9020, 0.9020],
         [0.5961, 0.5922, 0.5961,  ..., 0.8941, 0.8941, 0.8941],
         [0.6000, 0.6000, 0.5961,  ..., 0.8784, 0.8784, 0.8784],
         ...,
         [0.4235, 0.4157, 0.3412,  ..., 0.4000, 0.3961, 0.3961],
         [0.4353, 0.4510, 0.4157,  ..., 0.4353, 0.4314, 0.4275],
         [0.4588, 0.3725, 0.4392,  ..., 0.5137, 0.5098, 0.5020]],

        [[0.7216, 0.7216, 0.7176,  ..., 0.8392, 0.8392, 0.8471],
         [0.7059, 0.7020, 0.6941,  ..., 0.8431, 0.8431, 0.8431],
         [0.6902, 0.6902, 0.6863,  ..., 0.8314, 0.8314, 0.

In [79]:
temp

array([[[182, 155, 111],
        [182, 155, 111],
        [181, 154, 110],
        ...,
        [214, 230, 219],
        [214, 230, 219],
        [216, 230, 219]],

       [[178, 153, 111],
        [177, 152, 110],
        [177, 152, 110],
        ...,
        [213, 228, 220],
        [213, 228, 220],
        [215, 228, 220]],

       [[174, 152, 116],
        [174, 152, 116],
        [173, 151, 115],
        ...,
        [212, 224, 218],
        [212, 224, 218],
        [214, 224, 218]],

       ...,

       [[118, 108, 108],
        [116, 106, 106],
        [ 97,  87,  87],
        ...,
        [ 85, 102, 129],
        [ 85, 102, 129],
        [ 83, 101, 130]],

       [[121, 111, 111],
        [125, 115, 115],
        [116, 106, 106],
        ...,
        [ 94, 111, 138],
        [ 92, 111, 138],
        [ 91, 109, 138]],

       [[127, 117, 117],
        [105,  95,  95],
        [122, 112, 112],
        ...,
        [112, 131, 158],
        [112, 131, 158],
        [110, 128, 157]]

In [71]:
temp = orgi.permute(1, 2, 0).numpy()
temp.shape

(168, 299, 3)

In [44]:
img, _ = next(iter(dataloader['predict']))

In [48]:
img[0].permute(1, 2, 0).numpy()

array([[[-0.20797835,  0.57018375,  1.1383404 ],
        [-0.31368673,  0.48260483,  0.93557805],
        [-0.21191593,  0.45064625,  1.0173155 ],
        ...,
        [ 1.8691988 ,  2.0138144 ,  1.8461598 ],
        [ 1.9131557 ,  2.0584106 ,  1.8904009 ],
        [ 1.8191464 ,  1.9999095 ,  1.8908963 ]],

       [[ 0.5839637 ,  0.845097  ,  0.79892963],
        [ 0.5029029 ,  0.7702324 ,  0.63907486],
        [ 0.6041815 ,  0.7476078 ,  0.6705123 ],
        ...,
        [-0.13399763, -0.3208934 , -0.71628064],
        [ 0.00779251, -0.1770412 , -0.5735735 ],
        [-0.36199754, -0.52516735, -0.90673465]],

       [[-0.58662784, -0.4506869 , -0.39386553],
        [-0.64247304, -0.48767748, -0.53055185],
        [-0.8079642 , -0.7416179 , -0.8190527 ],
        ...,
        [ 0.10339201,  0.04532473,  0.0323505 ],
        [ 0.1142581 ,  0.05634901,  0.04328693],
        [ 0.08105387,  0.10624593,  0.02206189]],

       ...,

       [[-0.5609958 , -0.5230163 , -0.14613813],
        [-0

In [ ]:
# import os
# os.getcwd()